In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import catboost
from catboost import CatBoostRegressor 
import shap
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import graphviz
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.utils import plot_model
import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Classification Analysis

In [ ]:
# Read ECFP4 and store in dataframe
Data_04 = pd.read_csv(r"data_CHEMBL203-ECFP4.csv") 

# Classification Task
# add additional column for class label: pXC50 > 6 class 1 otherwise class 0
Class = Data_04['pXC50'] >= 6
Classes = []
for i in Class:
    if i == True:
        Classes.append(1)
    elif i == False:
        Classes.append(0)

Data_04['Class'] = Classes
X04 = Data_04.drop(['Smiles','pXC50','Class'],axis=1)
Y04 = Data_04['Class'].values.reshape(-1,1)

In [ ]:
# Apply SMOTE to balance Dataset
smote = SMOTE(sampling_strategy='minority',random_state=42)
X_sm, y_sm = smote.fit_resample(X, Y)

# check class count
y_s = list(Y_train) + list(Y_val) + list(Y_test)
pd.Series(y_s).value_counts()

In [ ]:
#Split data to trainin, validation and testing 
X_train, X_tes, Y_train, Y_tes = train_test_split(X_sm, y_sm, test_size=0.2,random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_tes, Y_tes, test_size=0.5,random_state=42)

In [ ]:
# Develop and train Catboost classification model using training and validation sets

params = {'iterations':3000,
        'learning_rate':0.01,
        'depth':7,
        'eval_metric':'Accuracy',
        'verbose':200,
        'od_type':"Iter", # overfit detector
        'od_wait':1000, # most recent best iteration to wait before stopping
        'random_seed': 2}

cat_model = CatBoostClassifier(**params)
cat_model.fit(X_train, Y_train,   
          eval_set=(X_val, Y_val), 
          use_best_model=False, # True if we don't want to save trees created after iteration with the best validation score
          plot=True);

In [ ]:
print(metrics.classification_report(Y_test,cat_model.predict(X_test))) # Evaluate Catboost model using test set

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10,5))
plt.xticks(size=17,weight='bold')
plt.yticks(size=17,weight='bold')
plt.title('Confusion Matrix for Catboost Algorithm',size=20,weight='bold')
sns.heatmap(confusion_matrix(Y_test, cat_model.predict(X_test)),annot=True,fmt='g',annot_kws={"size": 20,'weight':'bold'})
plt.savefig('Catboost CM 04',dpi=100,bbox_inches='tight')

# SHAP Analysis

In [ ]:
all_preds = cat_model.predict(X_test)
X_df = pd.DataFrame(X_test)
x_df = X_df.copy(deep=True)
x_df_1st = x_df.copy(deep=True)
x_df_1st['1st'] = all_preds
x_df = x_df.reset_index().drop('index',axis=1)
x_df_1st = x_df_1st.reset_index().drop('index',axis=1)
shap_values = shap.TreeExplainer(cat_model).shap_values(x_df) # Apply Shap
#shap.summary_plot(shap_values, x_df)
'Effect of input parameters on Pmax'
shap.summary_plot(shap_values, x_df,plot_size=(10,10),show=False,plot_type='dot',max_display=10)
plt.title('SHAP for CatBoost',weight='bold',size=20)
plt.xticks(size=20,weight='bold')
plt.yticks(size=20,weight='bold')
plt.savefig('SHAP 04Classify',dpi=100,bbox_inches='tight')

shap_values = shap.TreeExplainer(cat_model).shap_values(x_df)
feature_imp = np.mean(np.abs(shap_values),axis=0)
feature_imp.shape

# top 10 important features
ind = feature_imp.argsort()[-10:]
ind = ind[::-1] #arranging in descending order

In [ ]:
np.array(x_df.columns)[ind]
feature_imp[ind]
plt.figure(figsize=(10,8))
plot = sns.barplot(x=np.array(x_df.columns)[ind],y=feature_imp[ind],color=[0.1,0.2,0.1],order=ind)
plot.set_xticklabels(plot.get_xticklabels(), horizontalalignment='center',size=12)
plt.yticks(size=15,weight='bold')
plt.xticks(size=20,rotation=20,weight='bold')
plt.ylabel('Shap feature absolute importance',size=15,weight='bold')
plt.savefig('SHAP Feature Importance CB 04',dpi=100,bbox_inches='tight')

In [ ]:
# Define the CNN model architecture with regularization and dropout
CNN = Sequential()
CNN.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(1024, 1), kernel_regularizer=l2(0.001)))
CNN.add(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.001)))
CNN.add(MaxPooling1D(pool_size=2))
CNN.add(Flatten())
CNN.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
CNN.add(Dropout(0.5))
CNN.add(Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy loss and Adam optimizer
CNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Add early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
with open('CNN_Classification.txt', 'w') as f:
    CNN.summary(print_fn=lambda x: f.write(x + '\n'))
# Print the model summary
print(CNN.summary())
keras.utils.plot_model(CNN, "my_first_model.png",dpi=300)

# Fit the model with early stopping and validation data
history = CNN.fit(X_train, Y_train, epochs=100, validation_data=(X_val, Y_val), callbacks=[early_stopping])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
# Plot the training and validation loss
plt.plot(history.history['loss'],linewidth=5,linestyle='-')
plt.plot(history.history['val_loss'],linewidth=5,linestyle='-')
plt.title('CNN Model Loss',size=20,weight='bold')
plt.xticks(size=15,weight='bold')
plt.yticks(size=15,weight='bold')
plt.ylabel('Loss',size=20,weight='bold')
plt.xlabel('Epoch',size=20,weight='bold')
plt.legend(['Train', 'Validation'], loc='upper right',prop={"size":15,'weight':'bold'})
plt.savefig('CNN Loss 04',dpi=100,bbox_inches='tight')
plt.show()


plt.figure(figsize=(8,5))
# Plot the training and validation accuracy
plt.plot(history.history['accuracy'],linewidth=5,linestyle='-')
plt.plot(history.history['val_accuracy'],linewidth=5,linestyle='-')
plt.title('CNN Model Accuracy',size=20,weight='bold')
plt.xticks(size=15,weight='bold')
plt.yticks(size=15,weight='bold')
plt.ylabel('Accuracy',size=20,weight='bold')
plt.xlabel('Epoch',size=20,weight='bold')
plt.legend(['Train', 'Validation'], loc='lower right',prop={"size":15,'weight':'bold'})
plt.savefig('CNN Accuracy 04,dpi=100,bbox_inches='tight')
plt.show()

In [ ]:
# Model Evaluation and Visualization
preds = CNN.predict(X_test)
y_pred_binary = (preds >= 0.5).astype(int)
print(metrics.classification_report(Y_test,y_pred_binary))

# plot confusion matric for CNN
plt.figure(figsize=(10,5))
plt.xticks(size=17,weight='bold')
plt.yticks(size=17,weight='bold')
plt.title('Confusion Matrix for CNN',size=20,weight='bold')
sns.heatmap(confusion_matrix(Y_test, y_pred_binary),annot=True,fmt='g',cmap='Blues',annot_kws={"size": 20,'weight':'bold'})
plt.savefig('CNN CM 04',dpi=100,bbox_inches='tight')

In [ ]:
# Import Libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

In [ ]:
# Define the ANN model architecture with regularization and dropout
ANN = Sequential()
ANN.add(Dense(128, input_dim=1024, activation='relu', kernel_regularizer=l2(0.001)))
ANN.add(Dropout(0.5))
ANN.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
ANN.add(Dropout(0.5))
ANN.add(Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy loss and Adam optimizer
ANN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Add early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')


with open('ANN_Classification.txt', 'w') as f:
    ANN.summary(print_fn=lambda x: f.write(x + '\n'))

# Print the model summary
print(ANN.summary())

# Fit the model to the training data with early stopping and validation data
history2 = ANN.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test, Y_test), callbacks=[early_stopping])

# Evaluate the performance of the model on the test data
loss, accuracy = ANN.evaluate(X_test, Y_test, verbose=1)
print("Accuracy:", accuracy)

In [ ]:
# Visualizing Results
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
# Plot the training and validation loss
plt.plot(history2.history['loss'],linewidth=5,linestyle='-')
plt.plot(history2.history['val_loss'],linewidth=5,linestyle='-')
plt.title('ANN Model Loss',size=20,weight='bold')
plt.xticks(size=15,weight='bold')
plt.yticks(size=15,weight='bold')
plt.ylabel('Loss',size=20,weight='bold')
plt.xlabel('Epoch',size=20,weight='bold')
plt.legend(['Train', 'Validation'], loc='upper right',prop={"size":15,'weight':'bold'})
plt.savefig('ANN Loss 04',dpi=100,bbox_inches='tight')
plt.show()


plt.figure(figsize=(8,5))
# Plot the training and validation accuracy
plt.plot(history2.history['accuracy'],linewidth=5,linestyle='-')
plt.plot(history2.history['val_accuracy'],linewidth=5,linestyle='-')
plt.title('ANN Model Accuracy',size=20,weight='bold')
plt.xticks(size=15,weight='bold')
plt.yticks(size=15,weight='bold')
plt.ylabel('Accuracy',size=20,weight='bold')
plt.xlabel('Epoch',size=20,weight='bold')
plt.legend(['Train', 'Validation'], loc='lower right',prop={"size":15,'weight':'bold'})
plt.savefig('ANN Accuracy 04',dpi=100,bbox_inches='tight')
plt.show()

In [ ]:
# Model Evaluation and Visualization
preds = ANN.predict(X_test)
y_pred_binary = (preds >= 0.5).astype(int)
print(metrics.classification_report(Y_test,y_pred_binary))

# Plot confusion matrix for ANN
plt.figure(figsize=(10,5))
plt.xticks(size=17,weight='bold')
plt.yticks(size=17,weight='bold')
plt.title('Confusion Matrix for ANN',size=20,weight='bold')
sns.heatmap(confusion_matrix(Y_test, y_pred_binary),annot=True,fmt='g',cmap='Blues',annot_kws={"size": 20,'weight':'bold'})
plt.savefig('ANN CM 04',dpi=100,bbox_inches='tight')

In [ ]:
# Define the random forest classifier with 100 trees
RF = RandomForestClassifier(n_estimators=1000,max_depth=10)

# Fit the model to the training data
RF.fit(X_train, Y_train)

# Predict on the test data
y_pred = RF.predict(X_test)


# Display classification report and plot RF confusion matrix
print(metrics.classification_report(Y_test,RF.predict(X_test)))

In [ ]:
# Data Visualization
plt.figure(figsize=(10,5))
plt.xticks(size=17,weight='bold')
plt.yticks(size=17,weight='bold')
plt.title('Confusion Matrix for Random Forest',size=20,weight='bold')
sns.heatmap(confusion_matrix(Y_test, RF.predict(X_test)),annot=True,fmt='g',cmap='Blues',annot_kws={"size": 20,'weight':'bold'})
plt.savefig('RF CM 04',dpi=100,bbox_inches='tight')

In [ ]:
# Define the SVM model with a linear kernel
SVC = svm.SVC(kernel='rbf',class_weight='balanced', probability=True)

# Fit the model to the training data
SVC.fit(X_train, Y_train)

# Predict on the test data
y_pred = SVC.predict(X_test)

In [ ]:
# Display classification report and plot SVM confusion matrix
print(metrics.classification_report(Y_test,SVC.predict(X_test)))

In [ ]:
plt.figure(figsize=(10,5))
plt.xticks(size=17,weight='bold')
plt.yticks(size=17,weight='bold')
plt.title('Confusion Matrix for SVC',size=20,weight='bold')
sns.heatmap(confusion_matrix(Y_test, SVC.predict(X_test)),annot=True,fmt='g',cmap='Blues',annot_kws={"size": 20,'weight':'bold'})
plt.savefig('SVC CM 04',dpi=100,bbox_inches='tight')

# Regression Analysis

In [ ]:
# Regression Task
X_Pos = Data_Positive.drop(['Smiles','Class','pXC50'],axis=1)
Y_Pos = Data_Positive['pXC50'].values.reshape(-1,1)

Y_Pos = pd.Series(Y_Pos.reshape(1,-1)[0])

new_clm_heads = [x for x in range(0,len(X_Pos.columns))]
X_Pos.columns = new_clm_heads

Data_Positive = Data_04[Data_04['pXC50']>=6]
Data_Positive

In [ ]:
# Split data with active molecules to training, validation and testing
X_train, X_tes, Y_train, Y_tes = train_test_split(X_Pos, df, test_size=0.1, random_state=42,stratify=df.iloc[:,1])
X_val, X_test, Y_val, Y_test = train_test_split(X_tes, Y_tes, test_size=0.5, random_state=42,stratify=Y_tes.iloc[:,1])

In [ ]:
# Develop Catboost model

params = {'iterations':5000,
        'learning_rate':0.01,
        'depth':8,
        'eval_metric':'R2',
        'verbose':200,
        'od_type':"Iter", # overfit detector
        'od_wait':1000, # most recent best iteration to wait before stopping
        'random_seed': 8
          }

cat_model = CatBoostRegressor(**params)
cat_model.fit(X_train, Y_train.iloc[:,0],   
          eval_set=(X_val, Y_val.iloc[:,0]), 
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True );

In [ ]:
# Model Evaluation
pred_train = cat_model.predict(X_train)
pred_val = cat_model.predict(X_val)
pred_test = cat_model.predict(X_test)

print(np.sqrt(np.mean((Y_train.iloc[:,0]-pred_train)**2)))
print(np.sqrt(np.mean((Y_val.iloc[:,0]-pred_val)**2)))
print(np.sqrt(np.mean((Y_test.iloc[:,0]-pred_test)**2)))
print('\n')
print(r2_score(Y_train.iloc[:,0],pred_train))
print(r2_score(Y_val.iloc[:,0],pred_val))
print(r2_score(Y_test.iloc[:,0],pred_test))

# SHAP Analysis

In [ ]:
#Apply  SHAP for Catboost
all_preds = cat_model.predict(X_test)

X_df = pd.DataFrame(X_test)

x_df = X_df.copy(deep=True)
x_df_1st = x_df.copy(deep=True)

x_df_1st['1st'] = all_preds

x_df = x_df.reset_index().drop('index',axis=1)
x_df_1st = x_df_1st.reset_index().drop('index',axis=1)


shap_values = shap.TreeExplainer(cat_model).shap_values(x_df)
#shap.summary_plot(shap_values, x_df)


'Effect of input parameters on Pmax'
shap.summary_plot(shap_values, x_df,plot_size=(10,10),show=False,plot_type='dot',max_display=10)
plt.title('SHAP for CatBoost',weight='bold',size=20)
plt.xticks(size=20,weight='bold')
plt.yticks(size=20,weight='bold')
plt.savefig('Reg SHAP 04',dpi=100,bbox_inches='tight')


shap_values = shap.TreeExplainer(cat_model).shap_values(x_df)
feature_imp = np.mean(np.abs(shap_values),axis=0)
feature_imp.shape

# top 10 important features
ind = feature_imp.argsort()[-10:]
ind = ind[::-1] #arranging in descending order

np.array(x_df.columns)[ind]
feature_imp[ind]

In [ ]:
# Plotting
plt.figure(figsize=(10,8))
plot = sns.barplot(x=np.array(x_df.columns)[ind],y=feature_imp[ind],color=[0.1,0.2,0.1])
plot.set_xticklabels(plot.get_xticklabels(), horizontalalignment='center',size=12)
plt.yticks(size=15,weight='bold')
plt.xticks(size=20,rotation=20,weight='bold')
plt.ylabel('Shap absolute feature importance',size=15,weight='bold')
plt.savefig('Reg SHAP Feature Importance CB 04',dpi=100,bbox_inches='tight')

In [ ]:
# Develop ANN model
ANN_reg = models.Sequential()
ANN_reg.add(layers.Dense(100, input_dim = 1024, activation='relu',kernel_regularizer=l2(0.01)))
ANN_reg.add(layers.Dense(200, activation='relu',kernel_regularizer=l2(0.01)))
ANN_reg.add(layers.Dense(300, activation='relu',kernel_regularizer=l2(0.01)))
ANN_reg.add(layers.Dense(100, activation='relu',kernel_regularizer=l2(0.01)))
ANN_reg.add(layers.Dense(10, activation='relu',kernel_regularizer=l2(0.01)))
ANN_reg.add(layers.Dense(1))
ANN_reg.compile(optimizer='adam',loss='mse',metrics=['mse'])
early_stopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1, mode='min')
callbacks=[early_stopping]

history2 = ANN_reg.fit(X_train, Y_train.iloc[:,0], epochs=500, batch_size=50,validation_data=(X_val, Y_val.iloc[:,0]))
pred_train = ANN_reg.predict(X_train)
pred_val = ANN_reg.predict(X_val)
pred_test = ANN_reg.predict(X_test)

In [ ]:
# Develop RF model           
rf_default = RandomForestRegressor(n_estimators=3000,max_depth=12)
rf_default.fit(X_train,Y_train.iloc[:,0])
print(rf_default.score(X_train,Y_train.iloc[:,0]))
print(rf_default.score(X_val,Y_val.iloc[:,0]))
print(rf_default.score(X_test,Y_test.iloc[:,0]))

pred_train = rf_default.predict(X_train)
pred_test = rf_default.predict(X_test)
pred_val = rf_default.predict(X_val)

# SHAP Analysis

In [ ]:
# Aplly SHAP for RF            
all_preds = rf_default.predict(X_test)

X_df = pd.DataFrame(X_test)

x_df = X_df.copy(deep=True)
x_df_1st = x_df.copy(deep=True)

x_df_1st['1st'] = all_preds

x_df = x_df.reset_index().drop('index',axis=1)
x_df_1st = x_df_1st.reset_index().drop('index',axis=1)


shap_values = shap.TreeExplainer(rf_default).shap_values(x_df)
#shap.summary_plot(shap_values, x_df)


'Effect of input parameters on Pmax'
shap.summary_plot(shap_values, x_df,plot_size=(10,10),show=False,plot_type='dot',max_display=10)
plt.title('SHAP for Random Forest',weight='bold',size=20)
plt.xticks(size=20,weight='bold')
plt.yticks(size=20,weight='bold')
plt.savefig('RF Reg SHAP 04',dpi=100,bbox_inches='tight')

shap_values = shap.TreeExplainer(rf_default).shap_values(x_df)
feature_imp = np.mean(np.abs(shap_values),axis=0)
feature_imp.shape

# top 10 important features
ind = feature_imp.argsort()[-10:]
ind = ind[::-1] #arranging in descending order

np.array(x_df.columns)[ind]
feature_imp[ind]

In [ ]:
# Plotting Results
plt.figure(figsize=(10,8))
plot = sns.barplot(x=np.array(x_df.columns)[ind],y=feature_imp[ind],color=[0.1,0.2,0.1])
plot.set_xticklabels(plot.get_xticklabels(), horizontalalignment='center',size=12)
plt.yticks(size=15,weight='bold')
plt.xticks(size=20,rotation=20,weight='bold')
plt.ylabel('Shap absolute feature importance',size=15,weight='bold')
plt.savefig('RF Reg SHAP Feature Importance CB 04',dpi=100,bbox_inches='tight')

In [ ]:
# Develop GBT model
            
Y_train=Y_train['Values']
Ada = GradientBoostingRegressor(random_state=42)
Ada.fit(X_train,Y_train)

Ada.score(X_test,Y_test)

# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [4,6,10],'n_estimators': [30,100,1000,3000],'learning_rate':[0.1,0.01,0.001]}

ada = GradientBoostingRegressor(random_state=42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = ada, param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2,scoring='r2')


# Fit the grid search to the data
grid_search.fit(X_train, Y_train)
grid_search.best_params_

best_grid_GBT = grid_search.best_estimator_

best_grid_GBT
best_grid_GBT.score(X_test,Y_test)

In [ ]:
# Develope GBT Model
Ada = GradientBoostingRegressor(n_estimators=3000,max_depth=12,random_state=42)
Ada.fit(X_train,Y_train.iloc[:,0])

Ada.score(X_test,Y_test.iloc[:,0])
pred_train = Ada.predict(X_train)
pred_test = Ada.predict(X_test)
pred_val = Ada.predict(X_val)

In [ ]:
# Develop SVR model
SVR = svm.SVR(kernel='rbf',degree=3,C=3)

SVR.fit(X_train,Y_train.iloc[:,0])
pred_train = SVR.predict(X_train)
pred_test = SVR.predict(X_test)
pred_val = SVR.predict(X_val)